In [1]:
import requests
import dotenv
import os, time
import pandas as pd
import glob, json

dotenv.load_dotenv()

True

In [2]:
scratch = os.getenv('SCRATCH')
hashtag_folder = f"{scratch}/tiktok/hashtag_data"

oak = os.getenv('OAK')

video_folder = f"{oak}/samori/tiktok/videos"
comments_folder = f"{oak}/samori/tiktok/comments"
comments_folder2 = f"{scratch}/samori/tiktok/comments"
os.path.exists(comments_folder)

True

In [3]:
def get_access_token(client_key, client_secret):
    
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }

    response = requests.post(endpoint_url, headers=headers, data=data)

    if response.status_code == 200:
        response_json = response.json()
        return response_json
        
    else:
        # If the request was not successful, print the error response JSON
        print("Error:", response.json())

In [4]:
def make_request_comments(endpoint_name, query_body, query_params, headers, save_folder):

    video_id = query_body["video_id"]
    prev_cursor = query_body["cursor"]
    endpoints_dict = {"videos": "https://open.tiktokapis.com/v2/research/video/query/",
                     "comments":"https://open.tiktokapis.com/v2/research/video/comment/list/"}
    endpoint = endpoints_dict[endpoint_name]

    # make post request
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)
    status_code = response.status_code

    if response.status_code != 200:
        try:
            response_json = response.json()
        except json.JSONDecodeError as e:
            response_json = "Error reading json"
            
        return video_id, False, prev_cursor, response_json, status_code 

    
    # extracting information for pagination
    data = response.json().get("data", {})
    
    has_more = data["has_more"]
    cursor = data["cursor"]

    # saving queried data
    records = data.get(endpoint_name, [])
    df = pd.DataFrame(records)
    if len(df) != 0:
        df.to_csv(f"{save_folder}/{endpoint_name}_{video_id}_{cursor}.csv", index=False)

    return video_id, has_more, cursor, len(df), status_code

In [5]:
def query_api_comments(endpoint_name, fields, cursor, video_id, save_folder, log_name, credentials):
    
    # getting credentials
    # client_key = os.getenv("CLIENT_KEY")
    # client_secret = os.getenv("CLIENT_SECRET")
    
    # credentials = get_access_token(client_key, client_secret)
    access_token = credentials["access_token"]
    token_type = credentials["token_type"]
    
    query_params = {"fields": fields}
    query_body = {"video_id":video_id, "max_count":100, "cursor":cursor}
    
    headers = {"Content-Type": "application/json",
               "Authorization": f"{token_type} {access_token}"}
    
    has_more = True

    # pagination loop
    while has_more:
        query_body.update({"cursor": cursor})
            
        video_id, has_more, cursor, samples, status_code = make_request_comments(endpoint_name, query_body, query_params, headers, save_folder)

        with open(f"./download_logs/download_logs_{log_name}", "a") as f:
            f.write(f"{cursor}\t{has_more}\t{samples}\t{video_id}\t{status_code}\n")

        print(f"{cursor}\t{has_more}\t{samples}\t{video_id}\t{status_code}")
        time.sleep(5)
        
    return status_code

In [6]:
def get_comments(video_ids, endpoint_name, fields, cursor, save_folder, log_name):

    client_key = os.getenv("CLIENT_KEY")
    client_secret = os.getenv("CLIENT_SECRET")
    
    credentials = get_access_token(client_key, client_secret)

    
    downloaded_comments = os.listdir(save_folder)
    downloaded_video_ids = [i.split('_')[1] for i in downloaded_comments]

    col_names = ["cursor", "has_more","samples","video_id","status_code"]

    all_log_ids = []
    logs = glob.glob(f'./download_logs/download_logs_run_09032024_*')
    for log in logs:
        log_data = pd.read_csv(log, sep='\t', names=col_names, header=None)
        log_ids = list(log_data['video_id'])
        log_ids = [str(i) for i in log_ids]
        
        all_log_ids += log_ids
    
    vid_ids = downloaded_video_ids + all_log_ids
    vid_ids = list(set(vid_ids))
    
    status_code = 200
    for video_id in video_ids:
        if str(video_id) not in vid_ids:
            # continue
            if status_code != 429:
                if status_code == 401:
                    credentials = get_access_token(client_key, client_secret)
                    status_code = query_api_comments(endpoint_name, fields, cursor, video_id, save_folder, log_name, credentials)
                else:
                    status_code = query_api_comments(endpoint_name, fields, cursor, video_id, save_folder, log_name, credentials)
            else:
                break

In [12]:
video_data = pd.read_csv(f"{video_folder}/combined/all_months.csv")
video_ids = list(video_data['id'])

endpoint_name = "comments"
fields = "id,video_id,text,like_count,reply_count,parent_comment_id,create_time"
cursor = 0
save_folder = f"{comments_folder}/downloads"
log_name = 'run_09032024_28.txt'

get_comments(video_ids, endpoint_name, fields, cursor, save_folder, log_name)

OSError: [Errno 122] Disk quota exceeded: '/oak/stanford/groups/rbaltman/samori/tiktok/comments/downloads/comments_7307387251049303342_13.csv'

In [7]:
def get_comments2(video_ids, endpoint_name, fields, cursor, save_folder, log_name):
    
    client_key = os.getenv("CLIENT_KEY")
    client_secret = os.getenv("CLIENT_SECRET")
    
    credentials = get_access_token(client_key, client_secret)

    # oak_save_dir = 
    downloaded_comments = os.listdir(f"{comments_folder}/downloads")
    downloaded_video_ids = [i.split('_')[1] for i in downloaded_comments]

    col_names = ["cursor", "has_more","samples","video_id","status_code"]

    all_log_ids = []
    logs = glob.glob(f'./download_logs/download_logs_run_09032024_*')
    for log in logs:
        log_data = pd.read_csv(log, sep='\t', names=col_names, header=None)
        log_ids = list(log_data['video_id'])
        log_ids = [str(i) for i in log_ids]
        
        all_log_ids += log_ids
    
    vid_ids = downloaded_video_ids + all_log_ids
    vid_ids = list(set(vid_ids))
    
    status_code = 200
    for video_id in video_ids:
        if str(video_id) not in vid_ids:
            # continue
            if status_code != 429:
                if status_code == 401:
                    credentials = get_access_token(client_key, client_secret)
                    status_code = query_api_comments(endpoint_name, fields, cursor, video_id, save_folder, log_name, credentials)
                else:
                    status_code = query_api_comments(endpoint_name, fields, cursor, video_id, save_folder, log_name, credentials)
            else:
                break

In [ ]:
video_data = pd.read_csv(f"{video_folder}/combined/all_months.csv")
video_ids = list(video_data['id'])

endpoint_name = "comments"
fields = "id,video_id,text,like_count,reply_count,parent_comment_id,create_time"
cursor = 0
save_folder = f"{comments_folder2}/downloads"
log_name = 'run_09032024_34.txt'

get_comments2(video_ids, endpoint_name, fields, cursor, save_folder, log_name)

1	False	1	7205131472284568874	200
0	False	0	7205124342982774058	200
0	False	{'error': {'code': 'internal_error', 'message': 'Something went wrong. Please try again later.', 'log_id': '202412162144383D3C6F61E2D68B13B34D'}}	7205098328151985454	500
5	False	5	7205087570357800235	200
0	False	0	7205083855966145838	200
4	False	4	7205082564976217390	200
90	True	90	7205080503022636330	200
165	False	75	7205080503022636330	200
0	False	0	7205072259847884075	200
0	False	0	7205068449540787502	200


In [7]:
video_data = pd.read_csv(f"{video_folder}/combined/all_months.csv")
video_ids = list(video_data['id'])

endpoint_name = "comments"
fields = "id,video_id,text,like_count,reply_count,parent_comment_id,create_time"
cursor = 0
save_folder = f"{comments_folder}/downloads"
log_name = 'run_09032024_13.txt'

get_comments(video_ids, endpoint_name, fields, cursor, save_folder, log_name)

11	False	11	7356742697576631594	200
1	False	1	7356738510310608170	200
1	False	1	7356737177180458286	200
3	False	3	7356736627420499242	200
0	False	0	7356735486838181162	200
0	False	0	7356735433868430635	200
3	False	3	7356732960017501482	200
46	False	46	7356730480114355498	200
0	False	0	7362201209165352234	200
0	False	0	7362201105918332202	200
0	False	0	7362200701944106286	200
0	False	0	7362200548541599018	200
1	False	1	7362196109470502186	200
6	False	6	7362195795044470058	200
1	False	1	7362191794756455722	200
4	False	4	7362191439226359083	200
1	False	1	7362182634677554463	200
19	False	19	7362179176864025899	200
0	False	0	7362179105330171182	200
1	False	1	7362176926959750446	200
0	False	0	7362170608005795114	200
0	False	0	7362169125931666731	200
6	False	6	7362169090158431530	200
3	False	3	7362167458494565674	200
0	False	0	7362167454157524267	200
2	False	2	7362162103693774126	200
0	False	0	7362159698373791022	200
0	False	0	7362154329848941866	200
0	False	0	7362147439073086762	200
1	False	

AttributeError: 'Response' object has no attribute 'JSONDecodeError'

In [14]:
video_data = pd.read_csv(f"{video_folder}/combined/all_months.csv")
video_ids = list(video_data['id'])

endpoint_name = "comments"
fields = "id,video_id,text,like_count,reply_count,parent_comment_id,create_time"
cursor = 0
save_folder = f"{comments_folder}/downloads"
log_name = 'run_09032024_5.txt'

get_comments(video_ids, endpoint_name, fields, cursor, save_folder, log_name)

0	False	0	7353643222201158955	200
13	False	13	7353642921926692139	200
1	False	1	7353635820064263470	200
0	False	{'error': {'code': 'invalid_params', 'message': '`video_id: 7353631741648669995` is invalid. Please provide a valid video id', 'log_id': '20241017202752B462D9FBE2678412E6B3'}}	7353631741648669995	400
0	False	0	7353627599358201118	200
0	False	0	7353627114639346974	200
25	False	25	7353626514191093023	200
4	False	4	7353621808924445983	200
37	False	37	7353582046507273515	200
0	False	0	7353566508049501486	200
1	False	1	7353558183295683870	200
0	False	0	7353548367609204010	200
0	False	0	7353509979716078894	200
0	False	0	7353501583612611871	200
1	False	1	7353456603556777262	200
0	False	0	7353450018466057514	200
0	False	0	7353440075541237035	200
100	True	100	7353436795041893674	200
118	False	18	7353436795041893674	200
1	False	1	7353430553296768299	200
0	False	0	7353428947998100779	200
0	False	{'error': {'code': 'invalid_params', 'message': '`video_id: 7353352574403218731` is invalid.

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
video_data = pd.read_csv(f"{video_folder}/combined/all_months.csv")
video_ids = list(video_data['id'])

endpoint_name = "comments"
fields = "id,video_id,text,like_count,reply_count,parent_comment_id,create_time"
cursor = 0
save_folder = f"{comments_folder}/downloads"
log_name = 'run_09032024_2.txt'

get_comments(video_ids, endpoint_name, fields, cursor, save_folder, log_name)

0	False	0	7353791943429770539	200
0	False	0	7353781370864389419	200
0	False	0	7353772001997180203	200
0	False	0	7353767289692540206	200
0	False	0	7353753629821504814	200
0	False	{'error': {'code': 'invalid_params', 'message': '`video_id: 7353732441036639531` is invalid. Please provide a valid video id', 'log_id': '2024090404524443DCC7D5477901036B41'}}	7353732441036639531	400
0	False	0	7353725875793186078	200
0	False	0	7353722121484438815	200
0	False	0	7353682736902638891	200
0	False	0	7353676327196577067	200
0	False	0	7353664646781488427	200
0	False	0	7353632498397531434	200
0	False	0	7353629602213858590	200
0	False	0	7353625120100945183	200
0	False	0	7353624952429497642	200
0	False	0	7353620852740607278	200
3	False	3	7353585154541112618	200
3	False	3	7353541620094143790	200
0	False	0	7353514240373689646	200
0	False	0	7353497828125199647	200
5	False	5	7353471565691948331	200
0	False	0	7353468492143725867	200
0	False	0	7353467643871005998	200
0	False	0	7353456224433753386	200
1	False	1	

In [9]:
len(video_ids)

52047